# 数値シミュレーション

## 数値シミュレーションとは
　物理学とは、この宇宙を記述する学問である。そして、この宇宙は微分方程式で記述されている。方程式さえわかれば、数値的に解くことが可能である。よってある現象に着目した際の方程式を「支配方程式」と言い、その振る舞いを調べることを「数値シミュレーション」という。

　離散化...連続的な表現を行うために、離散的な値に変換することを指す。動画のように、静止画像をコマ送りすることで表現しているため、機械は連続的な表現が難しい。

## 差分化
　微分を離散単位で近似することを指す。差分化により微分方程式を扱う手法を「差分法」と言い、1次近似の差分法を「オイラー法」という。

## ニュートンの運動方程式
　運動方程式とは、時間に関する連立常微分方程式である。ここで、2つの物体の間に距離の2乗に反比例する力を仮定すると、宇宙空間内で互いに重力で引き合う2つの星の運動を表す方程式となる。これを解くと、起動が楕円になること、面積速度一定速、調和の法則などのケプラーの三法則が導かれる。
　自然科学において、法則とは、実験や観測などで